In [ ]:
import os, sys
import numpy as np
import geopandas as gpd
from onstove import OnStove, DataProcessor, RasterLayer, VectorLayer

In [ ]:
snake = True  # Change this to False to run the notebook by yourself and not through snakemake

# Data processing

## 1. Create a data processor
First, we will create an instance of the `DataProcessor` object that will be used to add raw datasets, process them, and generate the required output datapackages for the `MCA` and `OnStove`. The `DataProcessor` object accepts three optional arguments `project_crs`, `cell_size`, and `output_directory`. The first defines the Coordinate Reference System (crs) to use in the project, this means that all datasets will be reprojected to match such crs. The second is used to define the desired cell size (i.e. width and height in meters) of the output datasets, which will be used to ensure that all output datasets match such cell size. The third is used as the output folder in which all results will be written, this parameter defaults to `output`.

In [ ]:
data = DataProcessor(project_crs=3857, cell_size=(1000, 1000))

if snake:
    data_folder = snakemake.params.gis_data
    output_directory = snakemake.params.out_dir
else:
    data_folder = '../1. Data'
    output_directory = '../4. Results/Processed GIS Data'
data.output_directory = output_directory

## 2. Add a mask layer (country boundaries) and a base layer
A vector layer can be added as a mask, which will be later used to mask/clip all output datasets to the layer boundaries. For this, we use the `add_mask_layer` method providing a `category` in which to group the data, a `name` for the layer, and a `path` from where to read the data.

In [ ]:
if snake:
    adm_path = snakemake.input.boundaries
else:
    adm_path = os.path.join(data_folder, 'Administrative boundaries', 'NPL_adm0_Nepal0.shp')
data.add_mask_layer(category='Administrative', name='Country_boundaries', path=adm_path)


A raster base layer is needed to make every output match its grid and extent. For this, two additional options need to be passed to the `add_layer` method:
* `base_layer`: if `True` the added layer will be considered as the base layer. 
* `resample`: this is the resampling method to be used when resampling this layer to the desired `cell_size` if a `cell_size` is provided.

## 3. Add GIS layers
Similarly, we can add data layers using the `add_layer` method. A `category`, layer `name`, and `path` also need to be provided. In addition, the following arguments can be passed:
* `layer_type`: this argument is required with two possible options `raster` or `vector`, we should pass either one according to the dataset you are adding. 
* `resample`: this defines what resampling method to use when changing the resolution of the raster. The change of resolution happens when the layer gets aligned with the base layer.

### Demographics
Here we add the datasets of the demographics category, being population, the urban / rural divide, and the relative wealth index.

In [ ]:
if snake:
    pop_path = snakemake.input.population
else:
    pop_path =  os.path.join(data_folder, 'Demographics', 'Population', 'HRSL', 'population_npl_2018-10-01.tif')
data.add_layer(category='Demographics', name='Population', 
               path=pop_path, layer_type='raster', resample='sum')

if snake:
    wealth_path = snakemake.input.wealth
else:
    wealth_path = os.path.join(data_folder, 'Demographics', 'Wealth Index', 'Relative Wealth Index.tif')
data.add_layer(category='Demographics', name='Wealth', 
               path=wealth_path,
               layer_type='raster', normalization='MinMax', inverse=False, resample='nearest')

In [ ]:
if snake:
    municipality_path = snakemake.input.municipalities
else:
    municipality_path =os.path.join(data_folder, 'Administrative boundaries', 'NPL_adm3_Palika0', 'NPL_adm3_Palika0.shp')
data.add_layer(category='Demographics', name='Urban_rural_divide', 
               path=municipality_path, layer_type='vector', distance_method='rasterize')

classes = {'Gaunpalika': 11, 'Nagarpalika': 21, 'Mahanagarpalika': 21,
           'Upamahanagarpalika': 11, 'Wildlife Reserve': 11, 'National Park': 11,
           'Watershed and Wildlife Reserve': 11, 'Hunting Reserve': 11,
           'Development Area': 11}
data.layers['Demographics']['Urban_rural_divide'].data['value'] = data.layers['Demographics']['Urban_rural_divide'].data['Type_GN'].map(classes)

### Biomass
Here we add the datasets related to the biomass category. These datasets are a raster layer showing the average forest height on every pixel, and a walking friction layer that will be used later to calculate the travel time to the nearest forest resource. We also perform an additional process to the forest height layer, to exclude everything below 5 meters in height and calculate a canopy cover over each pixel.

**Note:** for the fricrtion layer, we also pass an additional argument `base_layer=True`, telling the `DataProcessor` to use the layer's spatial grid as the basis for the analysis. We chose this layer to be used as the base, because of its grid covering well the study area.

In [ ]:
if snake:
    forest_path = snakemake.input.forest
else:
    forest_path = os.path.join(data_folder, 'Forest cover', 'Forest_height.tif')
data.add_layer(category='Biomass', name='Forest',
               path=forest_path, layer_type='raster', resample='sum')
data.layers['Biomass']['Forest'].data[data.layers['Biomass']['Forest'].data < 5] = 0
data.layers['Biomass']['Forest'].data[data.layers['Biomass']['Forest'].data >= 5] = 1
data.layers['Biomass']['Forest'].meta['nodata'] = 0
transform = data.layers['Biomass']['Forest'].calculate_default_transform(data.project_crs)[0]
factor = (data.cell_size[0] ** 2) / (transform[0] ** 2)

In [ ]:
if snake:
    friction_path = snakemake.input.walking_friction
else:
    friction_path = os.path.join(data_folder, 'Walking friction', '202001_Global_Walking_Only_Friction_Surface_2019_Cropped.tif')
data.add_layer(category='Biomass', name='Friction', path=friction_path, distance_method='travel_time',
               layer_type='raster', resample='average', window=True, base_layer=True)

### Electricity
Here we add all datasets related to the electricity category. These datasets are the medium voltage lines, the night time lights and minigrids locations and capacity.

#### Medium voltage lines

In [ ]:
if snake:
    mv_path = snakemake.input.mv_lines
else:
    mv_path = os.path.join(data_folder, 'Power network', 'MV-network', 'Nepal_DL0.shp')
data.add_layer(category='Electricity', name='MV_lines', 
               path=mv_path, layer_type='vector',
               query="Status == 'In Service' | Status == 'In Service (Private)'")

#### Night time lights

In [ ]:
if snake:
    ntl_path = snakemake.input.ntl
else:
    ntl_path = os.path.join(data_folder, 'Night Time Lights', 'VNL_v21_npp_2020_global_vcmslcfg_c202205302300.average_masked.dat_cropped.tif')
data.add_layer(category='Electricity', name='Night_time_lights', 
               path=ntl_path, layer_type='raster', resample='average')

#### Mini grids

In [ ]:
if snake:
    mg_points_path = snakemake.input.mg_points
else:
    mg_points_path = os.path.join(data_folder, 'Power network', 'MG-hydro', 'micro_hydropower.shp')
data.add_layer(category='Electricity', name='MG_points', 
               path=mg_points_path, layer_type='vector')

if snake:
    mg_access_path = snakemake.input.mg_capacity
else:
    mg_access_path = os.path.join(data_folder, 'Power network', 'MG-hydro', 'Municipalities with MG hydro', 'mg_hydro.geojson')
data.add_layer(category='Electricity', name='MG_access', 
               path=mg_access_path, layer_type='vector')
data.layers['Electricity']['MG_access'].data.rename({'Municipality': 'municipality', 'kW_constructed': 'capacity', 'HHs_constructed': 'households'}, inplace=True, axis=1)
data.layers['Electricity']['MG_access'].data = data.layers['Electricity']['MG_access'].data[['municipality', 'capacity', 'households', 'geometry']]

### LPG
Here we add all datasets related to LPG category. They are the travel time to the nearest urban center using a motorized mean, and the roads network.

In [ ]:
if snake:
    lpg_path = snakemake.input.traveltime_urban
else:
    lpg_path = os.path.join(data_folder, 'Traveltime', 'traveltime_to_urban_by_road_cropped.tif')
data.add_layer(category='LPG', name='LPG_traveltime', 
               path=lpg_path, layer_type='raster', resample='average')

if snake:
    roads_path = snakemake.input.roads
else:
    roads_path = os.path.join(data_folder, 'Roads', 'Road_Networks_of_Nepal_OSM0.shp')
data.add_layer(category='LPG', name='Roads', 
               path=roads_path, layer_type='vector')

### Biogas
Here we add all datasets related to the biogas category. The datasets include livestock counts of buffaloes, cattle, chickens, goats, pigs and sheeps. We also add an extra layer to account for temperature, as at low temperatures biogas production is not efficient.

In [ ]:
if snake:
    buffaloes = snakemake.input.buffaloes
    cattle = snakemake.input.cattle
    poultry = snakemake.input.poultry
    goats = snakemake.input.goats
    pigs = snakemake.input.pigs
    sheeps = snakemake.input.sheeps
else:
    buffaloes = os.path.join(data_folder, 'Global livestock', 'Buffaloes', '5_Bf_2015_Da.tif')
    cattle = os.path.join(data_folder, 'Global livestock', 'Cattle', '5_Ct_2015_Da.tif')
    poultry = os.path.join(data_folder, 'Global livestock', 'Chickens', '5_Ch_2015_Da.tif')
    goats =os.path.join(data_folder, 'Global livestock', 'Goats', '5_Gt_2015_Da.tif')
    pigs = os.path.join(data_folder, 'Global livestock', 'Pigs', '5_Pg_2015_Da.tif')
    sheeps = os.path.join(data_folder, 'Global livestock', 'Sheep', '5_Sh_2015_Da.tif')

for key, path in {'buffaloes': buffaloes,
                  'cattle': cattle,
                  'poultry': poultry,
                  'goats': goats,
                  'pigs': pigs,
                  'sheeps': sheeps}.items():
    data.add_layer(category='Biogas/Livestock', name=key, path=path,
                   layer_type='raster', resample='nearest', window=True, rescale=True)

In [ ]:
if snake:
    temperature = snakemake.input.temperature
else:
    temperature = os.path.join(data_folder, 'Temperature', 'TEMP.tif')
data.add_layer(category='Biogas', name='Temperature', path=temperature,
               layer_type='raster', resample='average', window=True)

## 4. Mask reproject and align all required layers
After reading all datasets, we will align and reproject all rasters to the base layer, reproject the vector layers, calculate all needed distance rasters, finish calculating the canopy cover, and save all processed datasets to the `output_directory`.

In [ ]:
data.align_layers(datasets='all')

data.reproject_layers(datasets={'Electricity': ['MG_points', 'MG_access', 'MV_lines'],
                                'LPG': ['Roads'],
                                'Demographics': ['Urban_rural_divide']})

data.get_distance_rasters(datasets={'Electricity': ['MG_points', 'MV_lines'],
                                    'LPG': ['Roads'],
                                    'Demographics': ['Urban_rural_divide']})

data.mask_layers(datasets='all')

In [ ]:
## Finish calculating the canopy cover
data.layers['Biomass']['Forest'].data = data.layers['Biomass']['Forest'].data / factor
data.layers['Biomass']['Forest'].data *= 100
data.layers['Biomass']['Forest'].data[data.layers['Biomass']['Forest'].data > 100] = 100

In [ ]:
data.save_datasets(datasets='all')